## TO DO

- Add code to convert labels to proper format
- Figure out how to do proper testing (code to pair up images randomly, predict their score using neural network, and then compare whether the ordinal match was right using our score)
- Figure out how to incoroporate regression output
- Rewrite of code

In [ ]:
import os
import shutil
import random

In [ ]:
origin = 'dataset'
dest = 'data_images'

for directory in os.listdir(origin):
    for filename in os.listdir(os.path.join(origin, directory)):
        shutil.copy(os.path.join(origin, directory, filename), dest)

In [ ]:
# TODO resize all images to standard size

from keras import applications
from keras.layers import Input

# If you want to specify input tensor
input_tensor = Input(shape=(160, 160, 3))
vgg_model = applications.VGG16(weights='imagenet',
                               include_top=False,
                               input_tensor=input_tensor)

# To see the models' architecture and layer names, run the following
vgg_model.summary()

In [ ]:
# Creating dictionary that maps layer names to the layers
layer_dict = dict([(layer.name, layer) for layer in vgg_model.layers])

# Getting output tensor of the last VGG layer that we want to include
x = layer_dict['block4_pool'].output

# Stacking a new simple convolutional network on top of it    
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)

# Creating new model. Please note that this is NOT a Sequential() model.
from keras.models import Model
custom_model = Model(input=vgg_model.input, output=x)

# Make sure that the pre-trained bottom layers are not trainable
for layer in custom_model.layers[:15]:
    layer.trainable = False

# Do not forget to compile it
custom_model.compile(loss='categorical_crossentropy',
                     optimizer='rmsprop',
                     metrics=['accuracy'])



In [ ]:
import csv, sqlite3
import math

con = sqlite3.connect("imagion.db")
cur = con.cursor()

table_cols = [i[0] for i in cur.execute("SELECT * FROM imagion").description]

def get_filenames():
    
    files_dict = {}
    
    filenames = cur.execute("SELECT filename, scale_qsc FROM imagion")
    
    for file_, int_score in filenames.fetchall():
        files_dict[file_] = int_score
        
    return files_dict

In [ ]:
files_dict = get_filenames()

In [ ]:
# split training and test images

random.seed(10)

keys = files_dict.keys()
split = int(len(files_dict.keys()) * 0.75)

random.shuffle(keys)

train_keys = keys[:split]
test_keys = keys[split:]

In [ ]:
img_rows = 224
img_cols = 224

def chunks(l, n):
    """Yield successive n-sized chunks from l"""
    for i in xrange(0, len(l), n):
        yield l[i:i+n]

def get_train_data(chunk, img_row, img_col):
    X_train = []
    Y_train = []
    
    try:
        for imgname in chunk:
            Y_train.append(files_dict[imgname])
            filename = 'data_images'+'/'+imgname+'.png'
            img = cv2.imread(filename)
            img = np.rollaxis(cv2.resize(img,(img_row,img_col)).astype(np.float32),2)
            X_train.append(img)

        X_train = np.asarray(X_train)
        Y_train = np.asarray(Y_train)
        
        return X_train,Y_train

    except:
        X_train=None
        Y_train=None
        return X_train,Y_train
    
def get_test_data(chunk, img_row, img_col):
    X_test = []
    Y_test = []
    
    try:
        for imgname in chunk:
            Y_test.append(files_dict[imgname])
            folder = imgname[:idx]
            filename = './data_images'+'/'+imgname+'.png'
            img = cv2.imread(filename)
            img = np.rollaxis(cv2.resize(img,(img_row,img_col)).astype(np.float32),2)
            X_test.append(img)

        X_test = np.asarray(X_test)
        Y_test = np.asarray(Y_test)
        return X_test,Y_test
    except:
        X_train=None
        Y_train=None
        return X_train,Y_train

def getTrainData(chunk,nb_classes,img_rows,img_cols):
    X_train,Y_train = get_train_data(chunk,img_rows,img_cols)
    if (X_train!=None and Y_train!=None):
        X_train/=255
    return (X_train,Y_train)

def getTestData(chunk,nb_classes,img_rows,img_cols):
    X_test,Y_test = get_test_data(chunk,img_rows,img_cols)
    if (X_test!=None and Y_test!=None):
        X_test/=255
    return (X_test,Y_test)



In [ ]:
chunk_size = 10

for chunk in chunks(train_keys, chunk_size):
    X_chunk,Y_chunk=getTrainData(chunk,nb_classes,img_rows,img_cols)
    
    if (X_chunk!=None and Y_chunk!=None):
                #for X_batch, Y_batch in datagen.flow(X_chunk, Y_chunk, batch_size=chunk_size):
                loss = custom_model.fit(X_chunk, Y_chunk, verbose=1, batch_size=batch_size, nb_epoch=1, show_accuracy=True, validation_data=(X_test,Y_test))
                instance_count+=chunk_size
                print instance_count
                if instance_count%100==0:
                    model.save_weights('basic_model.h5',overwrite=True)